In [6]:
import pandas as pd
import json
from pathlib import Path

# ファイルパス
review_path = Path("../data/raw/Movies_and_TV.jsonl")
meta_path = Path("../data/raw/meta_Movies_and_TV.jsonl")

def load_josnl(path,limit=None):
    data = []
    with open(path,"r") as f:
        for i ,line in enumerate(f):
            data.append(json.loads(line))
            if limit and i + 1 >= limit:
                break
        return pd.DataFrame(data)

review_df = load_josnl(review_path,limit=500000)
meta_df = load_josnl(meta_path,limit=500000)

# asin をキーにマージ
df = pd.merge(review_df, meta_df, on="parent_asin", how="inner")

# 中身を確認
df.head()

,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,features,description,price,images_y,videos,store,categories,details,bought_together,author
0,5.0,Five Stars,"Amazon, please buy the show! I'm hooked!",[],B013488XFS,B013488XFS,AGGZ357AO26RQZVRLGU4D4N52DZQ,1440385637000,0,True,...,"[IMDb 8.1, 2017, 10 episodes, X-Ray, HDR, UHD,...",[A con man (Giovanni Ribisi) on the run from a...,None,[{'360w': 'https://images-na.ssl-images-amazon...,[],None,"[Suspense, Drama]","{'Content advisory': ['Nudity', 'violence', 's...",None,NaN
1,5.0,Five Stars,My Kiddos LOVE this show!!,[],B00CB6VTDS,B00CB6VTDS,AGKASBHYZPGTEPO6LWZPVJWB2BVA,1461100610000,0,True,...,"[2014, 13 episodes, X-Ray, ALL]",[Follow the adventures of Arty and his sidekic...,None,[{'360w': 'https://images-na.ssl-images-amazon...,[],None,[Kids],{'Audio languages': ['English Dialogue Boost: ...,None,NaN
2,5.0,What Love Is...,"...isn't always how you expect it to be, but w...",[],B001H1SVZC,B001H1SVZC,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,1590639227074,0,True,...,None,None,None,[{'360w': 'https://images-na.ssl-images-amazon...,[],None,None,"{'Subtitles': ['None available'], 'Directors':...",None,NaN
3,5.0,QUIRKY TURNS TO HEARTSTRINGS,As you learn about the very unique characters ...,[],B06WVW16WY,B06WVW16WY,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,1586999747540,0,True,...,None,None,None,[{'360w': 'https://images-na.ssl-images-amazon...,[],None,None,"{'Content advisory': ['Foul language'], 'Audio...",None,NaN
4,5.0,Way better than the harsh reviews.,Our family loved the film. We have kids and th...,[],B07RXM26FG,B07RXM26FG,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1569734232700,0,True,...,None,None,None,[{'360w': 'https://images-na.ssl-images-amazon...,[],None,None,"{'Audio languages': ['English', 'English [Audi...",None,NaN


In [7]:
def extract_first_image(images):
    if isinstance(images, list) and len(images) > 0:
        return images[0]
    elif isinstance(images, str):
        return images
    return None

df["image_url"] = df["images_y"].apply(extract_first_image)

In [8]:
df_nlp = df[["title_y", "text", "average_rating", "rating_number", "description","image_url"]].copy()
df_nlp = df_nlp.rename(columns={"average_rating": "rating"})

In [9]:
df_nlp["rating_number"] = df_nlp["rating_number"].apply(
    lambda x: int(x.replace(",", "")) if isinstance(x, str) else x
)

In [10]:
df_nlp = df_nlp.dropna(subset=["title_y", "text", "rating", "rating_number"])

In [11]:
output_path = Path("../data/processed/movie_reviews.csv")
df.to_csv(output_path, index=False)

In [12]:
df = pd.read_csv("../data/processed/movie_reviews.csv")
print(df.columns.tolist())

/var/folders/k0/qx33rxhd69g02ggn8hkrrq3r0000gn/T/ipykernel_94798/3726232333.py:1: DtypeWarning: Columns (17,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/movie_reviews.csv")


['rating', 'title_x', 'text', 'images_x', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase', 'main_category', 'title_y', 'subtitle', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images_y', 'videos', 'store', 'categories', 'details', 'bought_together', 'author', 'image_url']


In [24]:
# image_urlの例を10個表示
print(df['image_url'].iloc[10])

{'360w': 'https://images-na.ssl-images-amazon.com/images/S/pv-target-images/2fd0b502fcd45941192bbf629db2932e63146f96ac481652d18f14793f353a23._RI_SX360_FMwebp_.jpg', '480w': 'https://images-na.ssl-images-amazon.com/images/S/pv-target-images/2fd0b502fcd45941192bbf629db2932e63146f96ac481652d18f14793f353a23._RI_SX480_FMwebp_.jpg', '720w': 'https://images-na.ssl-images-amazon.com/images/S/pv-target-images/2fd0b502fcd45941192bbf629db2932e63146f96ac481652d18f14793f353a23._RI_SX720_FMwebp_.jpg', '1080w': 'https://images-na.ssl-images-amazon.com/images/S/pv-target-images/2fd0b502fcd45941192bbf629db2932e63146f96ac481652d18f14793f353a23._RI_SX1080_FMwebp_.jpg', '1440w': 'https://images-na.ssl-images-amazon.com/images/S/pv-target-images/2fd0b502fcd45941192bbf629db2932e63146f96ac481652d18f14793f353a23._RI_SX1440_FMwebp_.jpg', '1920w': 'https://images-na.ssl-images-amazon.com/images/S/pv-target-images/2fd0b502fcd45941192bbf629db2932e63146f96ac481652d18f14793f353a23._RI_SX1920_FMwebp_.jpg', 'variant'